In [6]:
pip install streamlit


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
!pip uninstall pyzmq -y
!pip install pyzmq --force-reinstall --no-binary :all:



Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'error'


  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [429 lines of output]
      Ignoring cffi: markers 'implementation_name == "pypy"' don't match your environment
           ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
           --- ------------------------------------ 0.3/3.2 MB ? eta -:--:--
           ------------- -------------------------- 1.0/3.2 MB 3.1 MB/s eta 0:00:01
           ---------------- ----------------------- 1.3/3.2 MB 2.6 MB/s eta 0:00:01
           ------------------- -------------------- 1.6/3.2 MB 2.4 MB/s eta 0:00:01
           ------------------- -------------------- 1.6/3.2 MB 2.4 MB/s eta 0:00:01
           ------------------- -------------------- 1.6/3.2 MB 2.4 MB/s eta 0:00:01
           ----------------------- ---------------- 1.8/3.2 MB 1.4 MB/s eta 0:00:01
           -------------------------- ------------- 2.1/3.2 MB 1.2 MB/s eta 0:00:01
       

In [6]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load experimental or simulated data
uploaded_file = st.file_uploader("Upload a CSV file", type="csv")
if uploaded_file:
    df = pd.read_csv(uploaded_file)

    st.sidebar.title("Controls")
    filter_type = st.sidebar.selectbox("Filter type", ["None", "Moving Average", "Savitzky-Golay"])
    gas_min = st.sidebar.slider("Gas zone start (m)", min_value=int(df["Distance_m"].min()), max_value=int(df["Distance_m"].max()), value=300)
    gas_max = st.sidebar.slider("Gas zone end (m)", min_value=gas_min, max_value=int(df["Distance_m"].max()), value=600)

    # Filtering
    df["Filtered_alpha"] = df["Estimated_alpha"]
    if filter_type == "Moving Average":
        df["Filtered_alpha"] = df["Estimated_alpha"].rolling(window=7, center=True).mean()
    elif filter_type == "Savitzky-Golay":
        from scipy.signal import savgol_filter
        df["Filtered_alpha"] = savgol_filter(df["Estimated_alpha"], window_length=11, polyorder=2)

    # Metrics in gas region
    mask = (df["Distance_m"] >= gas_min) & (df["Distance_m"] <= gas_max)
    true_alpha = df["True_alpha"][mask]
    est_alpha = df["Filtered_alpha"][mask]
    relative_error = np.abs((est_alpha - true_alpha) / (true_alpha + 1e-10)) * 100
    st.write(f"**Mean Relative Error:** {np.nanmean(relative_error):.2f}%")
    st.write(f"**Standard Deviation:** {np.nanstd(relative_error):.2f}%")

    # Plot
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(df["Distance_m"], df["True_alpha"], label="True α")
    ax.plot(df["Distance_m"], df["Estimated_alpha"], '--', alpha=0.5, label="Raw Estimated α")
    ax.plot(df["Distance_m"], df["Filtered_alpha"], label=f"{filter_type} α")
    ax.axvspan(gas_min, gas_max, color='gray', alpha=0.2, label="Gas Region")
    ax.set_xlabel("Distance (m)")
    ax.set_ylabel("Absorption α [1/m]")
    ax.legend()
    ax.grid(True)
    st.pyplot(fig)


2025-06-26 14:30:53.946 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-26 14:30:53.946 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-26 14:30:53.957 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-26 14:30:53.958 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-26 14:30:53.959 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-26 14:30:53.960 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
